<a href="https://colab.research.google.com/github/sanbgos/Se-alesysistemas/blob/main/Dashboard_circuitos_el%C3%A9ctricos_potencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Se utliza Cloudflare
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install streamlit
!apt install nodejs
!npm install -g localtunnel
import os
import threading
import time

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Funciones
# -------------------------

def generar_onda_rectificada(f, R, C, t, tipo='R'):
    w = 2 * np.pi * f
    vin = np.sin(w * t)
    vout = np.abs(vin)
    if tipo == 'RC':
        dt = t[1] - t[0]
        i = [vout[0]/R]
        for k in range(1, len(t)):
            i_k = (vout[k] - C/dt*(i[k-1]*dt/C - vout[k-1])) / (1 + R*C/dt)
            i.append(i_k)
        i = np.array(i)
    else:
        i = vout / R
    return vin, i

def calcular_thd(y, fs):
    N = len(y)
    Y = np.fft.rfft(y)
    mag = np.abs(Y) / N
    freqs = np.fft.rfftfreq(N, 1/fs)

    thd = np.sqrt(np.sum(mag[1:]**2)) / mag[0]
    return thd * 100, freqs, mag

def calcular_fp(thd):
    return 1 / np.sqrt(1 + (thd/100)**2)

# -------------------------
# Streamlit UI
# -------------------------

st.set_page_config(page_title="Análisis THD y FP", layout="wide")
st.title("📈 Análisis de THD y Factor de Potencia")
st.markdown("Simulación para rectificador de onda completa con carga **R** y **RC**.")

# Parámetros
f = st.slider("Frecuencia de entrada [Hz]", 50, 200, 60)
R = st.slider("Resistencia (R) [Ohm]", 1, 100, 10)
C = st.slider("Capacitancia (C) [F]", 0.0, 0.5, 0.05)

# Simulación
fs = 10000
t = np.linspace(0, 3/f, int(fs * 3/f))

vin, iR = generar_onda_rectificada(f, R, C, t, tipo='R')
_, iRC = generar_onda_rectificada(f, R, C, t, tipo='RC')

thd_r, fx_r, mag_r = calcular_thd(iR, fs)
fp_r = calcular_fp(thd_r)

thd_rc, fx_rc, mag_rc = calcular_thd(iRC, fs)
fp_rc = calcular_fp(thd_rc)

# Mostrar resultados
col1, col2 = st.columns(2)

with col1:
    st.subheader("🔌 Carga R")
    st.line_chart(iR)
    st.text(f"THD: {thd_r:.2f}%")
    st.text(f"Factor de Potencia: {fp_r:.4f}")
    st.pyplot(plt.figure(figsize=(6,3)))
    plt.stem(fx_r, mag_r, basefmt=" ")
    plt.title("FFT carga R")
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("Magnitud")

with col2:
    st.subheader("🔋 Carga RC")
    st.line_chart(iRC)
    st.text(f"THD: {thd_rc:.2f}%")
    st.text(f"Factor de Potencia: {fp_rc:.4f}")
    st.pyplot(plt.figure(figsize=(6,3)))
    plt.stem(fx_rc, mag_rc, basefmt=" ")
    plt.title("FFT carga RC")
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("Magnitud")


In [ ]:
import os
import threading
import time

# Iniciar streamlit en segundo plano
def run_app():
    os.system("streamlit run app.py &")

threading.Thread(target=run_app).start()

# Esperar que inicie
time.sleep(5)

# Crear túnel con cloudflared
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate
